In [1]:
# A new architecture of transformer with a head where key, query, value and projections layers
# are replaced with multilayer feedforward networks
# Rework of: https://github.com/karpathy/ng-video-lecture


In [2]:
import os
import urllib.request

import torch
import torch.nn as nn
from torch.nn import functional as F


In [3]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


Using device: cuda


In [4]:
# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 3
n_layer = 2
dropout = 0.2
functions_expansion_factor = 2
projection_expansion_factor = 2
ffwd_expansion_factor = 2


In [5]:
torch.manual_seed(1337)


In [6]:
file_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
path = "../datasets/shakespeare_char"
file_path = f"{path}/input.txt"

if not os.path.isfile(file_path):
    os.makedirs(path, exist_ok=True)
    print("Downloading dataset...")
    urllib.request.urlretrieve(file_url, file_path)

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()


In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string


In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [10]:
def calc_loss(logits: torch.Tensor, targets: torch.Tensor):
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)
    loss = F.cross_entropy(logits, targets)
    return loss


In [11]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)
            loss = calc_loss(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [12]:
class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_in, n_out, factor, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, factor * n_in),
            nn.ReLU(),
            nn.Linear(factor * n_in, n_out),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor):
        return self.net(x)


In [13]:
class DeepHead(nn.Module):
    """one head of self-attention"""

    def __init__(
        self,
        block_size,
        n_embd,
        head_size,
        dropout,
        functions_expansion_factor,
    ):
        super().__init__()
        self.key = FeedFoward(n_embd, head_size, functions_expansion_factor, dropout)
        self.query = FeedFoward(n_embd, head_size, functions_expansion_factor, dropout)
        self.value = FeedFoward(n_embd, head_size, functions_expansion_factor, dropout)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x: torch.Tensor):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)  # (B,T,hs)
        q = self.query(x)  # (B,T,hs)
        # compute attention scores ("affinities")
        wei = (
            q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        )  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [14]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(
        self,
        block_size,
        n_embd,
        n_head,
        head_size,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
    ):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                DeepHead(
                    block_size, n_embd, head_size, dropout, functions_expansion_factor
                )
                for _ in range(n_head)
            ]
        )
        self.proj = FeedFoward(
            head_size * n_head, n_embd, projection_expansion_factor, dropout
        )

    def forward(self, x: torch.Tensor):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out


In [15]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(
        self,
        block_size,
        n_embd,
        n_head,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
        ffwd_expansion_factor,
    ):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(
            block_size,
            n_embd,
            n_head,
            head_size,
            dropout,
            functions_expansion_factor,
            projection_expansion_factor,
        )
        self.ffwd = FeedFoward(n_embd, n_embd, ffwd_expansion_factor, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [16]:
class DeepHeadTransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        block_size,
        n_embd,
        n_layer,
        n_head,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
        ffwd_expansion_factor,
    ):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[
                Block(
                    block_size,
                    n_embd,
                    n_head,
                    dropout,
                    functions_expansion_factor,
                    projection_expansion_factor,
                    ffwd_expansion_factor,
                )
                for _ in range(n_layer)
            ]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx: torch.Tensor):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        return logits

    def generate(self, idx: torch.Tensor, max_new_tokens: int):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


In [17]:
model = DeepHeadTransformerModel(
    vocab_size,
    block_size,
    n_embd,
    n_layer,
    n_head,
    dropout,
    functions_expansion_factor,
    projection_expansion_factor,
    ffwd_expansion_factor,
)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")


9.610049 M parameters


In [18]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [19]:
model.train()

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    logits = model(xb)
    loss = calc_loss(logits, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.2970, val loss 4.2921
step 500: train loss 2.1585, val loss 2.2132
step 1000: train loss 1.7321, val loss 1.8979
step 1500: train loss 1.5981, val loss 1.7843
step 2000: train loss 1.5377, val loss 1.7419
step 2500: train loss 1.4917, val loss 1.6926
step 3000: train loss 1.4635, val loss 1.6634
step 3500: train loss 1.4386, val loss 1.6486
step 4000: train loss 1.4162, val loss 1.6255
step 4500: train loss 1.3933, val loss 1.6057
step 5000: train loss 1.3807, val loss 1.6055
step 5500: train loss 1.3590, val loss 1.5965
step 6000: train loss 1.3493, val loss 1.5988
step 6500: train loss 1.3264, val loss 1.5820
step 7000: train loss 1.3220, val loss 1.5553
step 7500: train loss 1.3084, val loss 1.5517
step 8000: train loss 1.2946, val loss 1.5491
step 8500: train loss 1.2953, val loss 1.5454
step 9000: train loss 1.2801, val loss 1.5411
step 9500: train loss 1.2716, val loss 1.5410
step 10000: train loss 1.2668, val loss 1.5455
step 10500: train loss 1.2583, val lo

In [20]:
model.eval()

with torch.no_grad():
    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
    # open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))



all not a good better in preyment
shortic: yet it was not a barren; and, on breathe news
letter of them and have the stones of that doth beast, I speak
the weeping of the canor of love of will.

HORTENSIO:
Sririm her souls extermits to lie and one.

GREMIO:
Master BERLAND:
And most bosom of lawful heart
In so doth here to his if alleet them to make him
But slipp honourable and to him.

GREMIO:
There my mind is less rightny of him;
The monument issue of all one couple-steeling;
As for the reap of
